In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By


options = webdriver.EdgeOptions()
options.add_argument("--disable-blink-features=AutomationControlled") 
 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
options.add_experimental_option("useAutomationExtension", False) 

driver = webdriver.Edge(options=options)

driver.get('https://www.energymarketprice.com/home/en/login')
time.sleep(10)
username = driver.find_element(By.XPATH,'//*[@id="mat-input-0"]')
password = driver.find_element(By.XPATH,'//*[@id="mat-input-1"]')
username.send_keys('g.pickering@urbanchain.co.uk')
time.sleep(5)
password.send_keys('Empgarryuc2!')
login = driver.find_element(By.XPATH, '/html/body/emp-root/emp-authentication/div/div[2]/emp-login/form/div/button[1]')
login.click()
time.sleep(5)

contracts = ['UKFeb25', 'UKSum25', 'UKWin25', 'UKSum26', 'UKWin26', 'UKSum27']
market = [25243, 25039, 42059, 42187, 42181, 42193, 42202, 42199, 42205, 202954, 202953, 189826, 85276, 85258, 189838, 202958, 202957, 189828]
period = ['Peak','Base', 'Off-peak']
session = [(c, p) for c in contracts for p in period]

# One day prior
start = '20240101'
# Current day
end = '20250102'
current_start = '01-01-2024'
current_end = '02-01-2025'

for s, m in zip(session, market):
    url = f'https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName={s[0]}{s[1]}_{start}_{end}&Market={m}&DateFrom={current_start}&DateTo={current_end}'
    driver.get(url)
    print(url)
    time.sleep(5)


https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName=UKFeb25Peak_20240101_20250102&Market=25243&DateFrom=01-01-2024&DateTo=02-01-2025
https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName=UKFeb25Base_20240101_20250102&Market=25039&DateFrom=01-01-2024&DateTo=02-01-2025
https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName=UKFeb25Off-peak_20240101_20250102&Market=42059&DateFrom=01-01-2024&DateTo=02-01-2025
https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName=UKSum25Peak_20240101_20250102&Market=42187&DateFrom=01-01-2024&DateTo=02-01-2025
https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName=UKSum25Base_20240101_20250102&Market=42181&DateFrom=01-01-2024&DateTo=02-01-2025
https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName=UKSum25Off-peak_20240101_20250102&Market=42193&DateFrom=01-01-2024&DateTo=02-01-2025
https://www.energymarketprice.com/new/DChart.asp?src=dlexcel&FileName=UKWin25Peak_2024

In [5]:
import pandas as pd
import os
import xlrd
import csv
import win32com.client as win32

for c in contracts:
    for p in period:
        file = rf'C:\Users\ThomasChow\Downloads/{c}{p}_{start}_{end}'
        
        
        fname = file + '.xls'
        excel = win32.gencache.EnsureDispatch('Excel.Application')
        wb = excel.Workbooks.Open(fname)
        
        wb.SaveAs(fname+"x", FileFormat = 51)    #FileFormat = 51 is for .xlsx extension
        wb.Close()                               #FileFormat = 56 is for .xls extension
        excel.Application.Quit()
        os.remove(fname)

In [6]:
import dateutil.relativedelta
full = {'UKFeb25': 'February 25', 'UKSum25':'Summer 25', 'UKWin25': 'Winter 25', 'UKSum26':'Summer 26', 'UKWin26': 'Winter 26', 'UKSum27': 'Summer 27'}

df1 = pd.DataFrame([],
                   columns=['Date', 'Value', 'Currency/Unit', 'daily_diff', 'daily_percentage',
       'weekly_diff', 'weekly_percentage', 'monthly_diff',
       'monthly_percentage', 'contract', 'load'])

for c in contracts:
    for p in period:
        file = rf'C:\Users\ThomasChow\Downloads/{c}{p}_{start}_{end}.xlsx'
        df = pd.read_excel(file, sheet_name = 'Sheet1', skiprows = [0,1,2])
        df['daily_diff'] = df.Value[len(df)-1]-df.Value[len(df)-2]
        df['daily_percentage'] = df['daily_diff']/df.Value[len(df)-2]

        current_week = df[df.Date>=df.Date[len(df)-1] + dateutil.relativedelta.relativedelta(days=-7)]
        previous_week = df[(df.Date<df.Date[len(df)-1] + dateutil.relativedelta.relativedelta(days=-7)) & (df.Date >= df.Date[len(df)-1] + dateutil.relativedelta.relativedelta(days=-14))]
        df['weekly_diff'] = current_week.agg({'Value':['mean'] }).Value[0] - previous_week.agg({'Value':['mean'] }).Value[0]
        df['weekly_percentage'] = df['weekly_diff']/previous_week.agg({'Value':['mean'] }).Value[0]

        current_month = df[df.Date>=df.Date[len(df)-1] + dateutil.relativedelta.relativedelta(months=-1)]
        previous_month = df[(df.Date<df.Date[len(df)-1] + dateutil.relativedelta.relativedelta(months=-1)) & (df.Date >= df.Date[len(df)-1] + dateutil.relativedelta.relativedelta(months=-2))]
        df['monthly_diff'] = current_month.agg({'Value':['mean'] }).Value[0] - previous_month.agg({'Value':['mean'] }).Value[0]
        df['monthly_percentage'] = df['monthly_diff']/previous_month.agg({'Value':['mean'] }).Value[0]

        df['contract'] = full[c]
        df['load'] = p
        df1 = pd.concat([df1, df])
        df1.reset_index(drop=True, inplace=True)
        os.remove(file)

df1.to_csv('forward.csv', index = False)

In [7]:
df1

,Date,Value,Currency/Unit,daily_diff,daily_percentage,weekly_diff,weekly_percentage,monthly_diff,monthly_percentage,contract,load
0,2024-01-02,108.800000,GBP/MWh,2.100000,0.017751,8.430000,0.077219,-4.084524,-0.034755,February 25,Peak
1,2024-01-03,112.300000,GBP/MWh,2.100000,0.017751,8.430000,0.077219,-4.084524,-0.034755,February 25,Peak
2,2024-01-04,111.700000,GBP/MWh,2.100000,0.017751,8.430000,0.077219,-4.084524,-0.034755,February 25,Peak
3,2024-01-05,113.650000,GBP/MWh,2.100000,0.017751,8.430000,0.077219,-4.084524,-0.034755,February 25,Peak
4,2024-01-08,107.950000,GBP/MWh,2.100000,0.017751,8.430000,0.077219,-4.084524,-0.034755,February 25,Peak
...,...,...,...,...,...,...,...,...,...,...,...
4641,2024-12-23,59.656843,GBP/MWh,1.180243,0.019111,2.219369,0.037278,-0.844557,-0.013995,Summer 27,Off-peak
4642,2024-12-26,60.287961,GBP/MWh,1.180243,0.019111,2.219369,0.037278,-0.844557,-0.013995,Summer 27,Off-peak
4643,2024-12-27,62.034279,GBP/MWh,1.180243,0.019111,2.219369,0.037278,-0.844557,-0.013995,Summer 27,Off-peak
4644,2024-12-30,61.757814,GBP/MWh,1.180243,0.019111,2.219369,0.037278,-0.844557,-0.013995,Summer 27,Off-peak
